In [0]:
%run "../setup/functions"

In [0]:
dbutils.widgets.text("start_date", "2020-04-22")
start_date = dbutils.widgets.get("start_date")

In [0]:
dbutils.widgets.text("terminate_date", "2020-04-27")
end_date = dbutils.widgets.get("terminate_date")

##Fetching Data from Bronze and Transforming

In [0]:
state = "california"
bronze_df = spark.read.format("json").load(f"{bronze_folder_path}{state.replace(' ', '')}")

In [0]:
filtered_df = bronze_df.filter(bronze_df.county=='Los Angeles')

In [0]:
display(filtered_df)

combined_name,confirmed,confirmed_daily,country,country_code,country_iso2,country_iso3,county,date,deaths,deaths_daily,loc,population,state,uid,CHAVE_DATA_DIA
"Los Angeles, California, US",20996,573,US,840,US,USA,Los Angeles,2020-04-28T00:00:00.000Z,1002,58,"List([-118.2282, 34.3083], Point)",10039107,California,84006037,20200428
"Los Angeles, California, US",22485,1489,US,840,US,USA,Los Angeles,2020-04-29T00:00:00.000Z,1056,54,"List([-118.2282, 34.3083], Point)",10039107,California,84006037,20200429
"Los Angeles, California, US",20423,856,US,840,US,USA,Los Angeles,2020-04-27T00:00:00.000Z,944,28,"List([-118.2282, 34.3083], Point)",10039107,California,84006037,20200427
"Los Angeles, California, US",18545,1008,US,840,US,USA,Los Angeles,2020-04-24T00:00:00.000Z,850,53,"List([-118.2282, 34.3083], Point)",10039107,California,84006037,20200424
"Los Angeles, California, US",19567,434,US,840,US,USA,Los Angeles,2020-04-26T00:00:00.000Z,916,21,"List([-118.2282, 34.3083], Point)",10039107,California,84006037,20200426
"Los Angeles, California, US",17537,1090,US,840,US,USA,Los Angeles,2020-04-23T00:00:00.000Z,797,65,"List([-118.2282, 34.3083], Point)",10039107,California,84006037,20200423
"Los Angeles, California, US",16447,1294,US,840,US,USA,Los Angeles,2020-04-22T00:00:00.000Z,732,69,"List([-118.2282, 34.3083], Point)",10039107,California,84006037,20200422
"Los Angeles, California, US",19133,588,US,840,US,USA,Los Angeles,2020-04-25T00:00:00.000Z,895,45,"List([-118.2282, 34.3083], Point)",10039107,California,84006037,20200425


In [0]:
df_selected = filtered_df.select(col('county'),
                             col('confirmed_daily'),
                             col('date'),
                             col('deaths_daily'))
                             

In [0]:
 results_final_df = df_selected\
  .withColumnRenamed("county", "MUNICIPIO")\
  .withColumnRenamed("confirmed_daily", "CONFIRMADOS_LA")\
  .withColumnRenamed("deaths_daily", "MORTES_LA")\
  .withColumn('CHAVE_DATA', date_format(col('date'),"yyyyMM"))\
  .withColumn('DIA', date_format(col('date'),"dd"))\
  .drop('date')
 
  

##Writing into Silver Layer

In [0]:
from delta.tables import DeltaTable
if (DeltaTable.isDeltaTable(spark, f"{silver_folder_path}{state.replace(' ', '')}")):
    deltaTable = DeltaTable.forPath(spark, f"{silver_folder_path}{state.replace(' ', '')}")
    deltaTable.alias("tgt").merge(
    results_final_df.alias("src"),
     "tgt.CHAVE_DATA = src.CHAVE_DATA AND tgt.DIA = src.DIA") \
    .whenMatchedUpdateAll()\
    .whenNotMatchedInsertAll()\
    .execute()
else:
    results_final_df.write.mode("overwrite")\
    .partitionBy("CHAVE_DATA")\
    .format("delta")\
    .save(f"{silver_folder_path}{state.replace(' ', '')}")

In [0]:
display(spark.read.format("delta").load(f"{silver_folder_path}{state.replace(' ', '')}"))

MUNICIPIO,CONFIRMADOS_LA,MORTES_LA,CHAVE_DATA,DIA
Los Angeles,573,58,202004,28
Los Angeles,1489,54,202004,29
Los Angeles,856,28,202004,27
Los Angeles,1008,53,202004,24
Los Angeles,434,21,202004,26
Los Angeles,1090,65,202004,23
Los Angeles,1294,69,202004,22
Los Angeles,588,45,202004,25
